In [6]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [7]:
# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone

# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


# Run Train_Flow

In [135]:
import train_flow
import importlib
importlib.reload(train_flow)

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['include_new']#['exclude_new', 'include_new', 'run_separately']
train_flow.pi_handling_test(pi_handlings)

"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""


# ---------- #
# UPDATE ME  #
# ---------- #
new_features = feature_dict['volu']
target_horizons = [2]#, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 120
test_day = 1
feature_cols = feature_dict['ma_lag'] #feature_cols
list_name = "ma_num+exp"


# --------- #
# LEAVE ME  #
# --------- # 
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
results_df, perm_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)

Running for horizon 2 | include_new
86 | 8 | All Cols: ['ADL', 'SMA_10_Lag200_min', 'VROC_10', 'VROC_3', 'VROC_5', 'Vol_Ratio_100', 'Vol_Ratio_25_zscore', 'Vol_Ratio_50_zscore']
Run 1/120 | Train: 2021-03-17 → 2026-01-30 | Test: 2026-02-04 → 2026-02-04 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
86 | 8 | All Cols: ['CMF_20', 'OBV_Z10', 'VROC_10', 'VROC_5', 'Vol_Ratio_100', 'Vol_Ratio_100_zscore', 'Vol_Ratio_25_zscore', 'Vol_Ratio_50_zscore']
Run 2/120 | Train: 2021-03-16 → 2026-01-29 | Test: 2026-02-03 → 2026-02-03 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
86 | 8 | All Cols: ['CMF_10', 'SMA_10_Lag200_min', 'VROC_10', 'VROC_3', 'VROC_5', 'Vol_Ratio_100_zscore', 'Vol_Ratio_25_zscore', 'Vol_Ratio_50_zscore']
Run 3/120 | Train: 2021-03-15 → 2026-01-28 | Test: 2026-02-02 → 2026-02-02 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
86 | 8 | All Cols: ['CMF_20', 'SMA_10_Lag200_min', 'VROC_10', 'VROC_5', 'Vol_Ratio_100', 'Vol_Ratio_25_zscore', 'Vol_Ratio_50_zscor

KeyboardInterrupt: 

# Run for combo of sets

In [ ]:
import train_flow
import importlib
importlib.reload(train_flow)
from itertools import combinations


# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['run_separately'] #'exclude_new', 'include_new', 'run_separately'
new_feature = past_ret_cols
train_flow.pi_handling_test(pi_handlings)
keys = list(feature_dict.keys())
two_combos = list(combinations(keys, 2))
new_feat_combos = [(k, new_feature) for k in keys]


"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""

master_results = []
master_perm = []

for a, b in new_feat_combos:
    # ---------- #
    # UPDATE ME  #
    # ---------- #
    #new_features = feature_dict[b] # use when referencing feature dict only
    new_features = b
    target_horizons = [2, 5, 10, 20, 30]
    train_years = [5]
    pi_years = [1.5]
    min_feats = [4]
    days_assessed = 120
    test_day = 1
    feature_cols = feature_dict[a] #feature_cols
    #list_name = f"{a}-{b}"
    list_name = f"{a}-pr"
    print(f"{a}-pr")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    results_df, perm_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, new_features)
    
    master_results.append(results_df)
    master_perm.append(perm_df)

# final master DataFrames
master_results_df = pd.concat(master_results, ignore_index=True)
master_perm_df    = pd.concat(master_perm, ignore_index=True)

ma_lag-['Past_Return%_1', 'Past_Return%_2', 'Past_Return%_3', 'Past_Return%_5', 'Past_Return%_10', 'Past_Return%_20', 'Past_Return%_30', 'Past_Return_1_sum10', 'Past_Return_2_sum10', 'Past_Return_3_sum10', 'Past_Return_5_sum10', 'Past_Return_10_sum10', 'Past_Return_20_sum10', 'Past_Return_30_sum10']
Running for horizon 2 | run_separately


KeyboardInterrupt: 

# Save most recent run

In [ ]:
# ---------- #
# UPDATE ME  #
# ---------- #
df = pd.read_csv(f"horizon_r_baseline.csv")


# --------- #
# LEAVE ME  #
# --------- #
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon',
        'pi_size', 'min_feats']
df_new = master_results_df[cols].copy()
df_concat = pd.concat([df[cols], df_new], ignore_index=True)


# ---------- #
# UPDATE ME  #
# ---------- #
df_concat.to_csv(f"horizon_r_baseline_new.csv", index=False)

# Run Performance_Flow

In [165]:
import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "horizon_2_baseline_new.csv" # Match prior cell saved as file name horizon_2_baseline_new
min_th = 0.55
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_main)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_main, perf_df, returns, min_th)

composite_score

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
27,2,xgboost-3,6,ma_num_ba,1.5,100,0.56,112.0,0.59,0.57,0.33,4.09,0.17,0.63,0.55,0.77,0.57,0.57,0.47
57,2,xgboost-3,6,experimental_slope_ba,1.5,6,0.54,114.0,0.58,0.56,0.33,3.48,0.15,0.59,0.57,0.82,0.60,0.52,0.46
3,2,xgboost-3,6,ma_num-baseline,1.5,8,0.56,111.0,0.58,0.55,0.34,3.85,0.15,0.62,0.53,0.78,0.56,0.55,0.45
65,2,xgboost-3,5,ma_lag-rsi_macd,1.5-r,4,0.55,105.0,0.57,0.64,0.32,3.36,0.14,0.60,0.54,0.74,0.65,0.62,0.45
4,2,xgboost-3,6,rsi_macd-baseline,1.5,8,0.56,104.0,0.57,0.54,0.32,2.14,0.14,0.62,0.52,0.84,0.63,0.45,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2,xgboost-3,5,ma_num-atr_adxvola,1.5-r,4,0.55,111.0,0.39,0.37,0.43,4.67,-0.22,0.47,0.30,0.77,0.51,0.23,0.24
1,2,xgboost-3,6,ma_rel-baseline,1.5,8,0.55,110.0,0.39,0.35,0.48,7.71,-0.22,0.46,0.31,0.79,0.44,0.27,0.23
51,2,xgboost-3,6,volu_ba,1.5,6,0.54,109.0,0.38,0.41,0.46,5.13,-0.23,0.45,0.32,0.77,0.44,0.37,0.23
23,2,xgboost-3,6,ma_rel_ba,1.5,100,0.55,113.0,0.38,0.35,0.48,6.99,-0.23,0.47,0.29,0.80,0.44,0.26,0.23
